<a href="https://colab.research.google.com/github/isaac-mackey/mind-uploading/blob/main/Autobiographical_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs and Imports

In [2]:
%%capture
!pip install openai

In [3]:
%%capture
import openai
from openai import OpenAI

## Autobiographical Prompts

In [ ]:
describe_prompt_list = [
    "Describe your childhood memories.",
    "Describe significant events or milestones from my early years.",
    "Describe your family members and their influence on your life.",
    "Describe important family gatherings or traditions.",
    "Describe your academic achievements and challenges.",
    "Describe pivotal moments in your education."
    "Describe pivotal moments in your career path.",
    "Describe your career progression and professional achievements.",
    "Describe your job changes, promotions, or significant projects.",
    "Describe your personal goals and aspirations.",
    "Describe significant life events.",
    "Describe how major life events have shaped your identity and values.",
    "Describe your physical and mental health.",
    "Describe your hobbies and passions.",
    "Describe your creative projects.",
    "Describe your travel experiences."
    "Describe your important friendships and social connections.",
    "Describe memorable social gatherings or meaningful relationships.",
    "Describe your core values and beliefs.",
    "Describe moral or ethical dilemmas you faced and how they were resolved."]

topics = ['childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel', 'friends', 'morals']

longer_topics = ['childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel', 'friends', 'morals', 'career', 'relationships', 'technology', 'art', 'culture', 'finances', 'sports', 'food', 'entertainment']

topic_prompt_list = ["Tell me about your "+x+"." for x in longer_topics]

factual_prompt_list = [
    "Identify ten biographical facts about yourself",
    "Identify ten biographical facts about yourself that will not change soon",
    "List some people who know you personally",
    "What is your full name?",
    "When and where were you born?",
    "Who are your parents and siblings?",
    "Where did you grow up?",
    "What schools did you attend?",
    "What was your childhood like?",
    "What were your favorite childhood activities?",
    "Did you have any childhood pets?",
    "What was your first job?",
    "What is your current job?",
    "Have you ever been married?",
    "Do you have any children?",
    "What are your hobbies?",
    "What are your favorite foods?",
    "What languages do you speak?",
    "What is your favorite book/movie?",
    "What is your favorite place you've visited?",
    "What is your biggest accomplishment?",
    "What are your future goals?",
    "What is your most memorable moment?"
]

## Get answers to prompts

In [ ]:
files_to_retrieve = []

sms_dated_txt_id = 'file-PyQ0LQpsZd8AqcXbJEOcASm1'
files_to_retrieve.append(sms_dated_txt_id)

journal_march_2011_jan_2018_txt_id = 'file-7yqmFEB36Ym7AorsSYhoTtmE'
files_to_retrieve.append(journal_march_2011_jan_2018_txt_id)

# old fine_tuned_model_id = 'ft:gpt-3.5-turbo-0613:university-of-california-santa-barbara:isaac-sms-bot:8ds3OPKx'
# fine_tuned_model_id = 'ft:gpt-3.5-turbo-0613:university-of-california-santa-barbara:isaac-sms-combined:8fD18uw4'

system_message = ("Welcome to Isaac's personal assistant! You now have access to information about Isaac's life. "
                  "Use this information to answer the prompts as if you are him. "
                  "Your goal is to demonstrate excellent autobiographical memory. "
                  "Don't tell me what you're doing, just answer the prompts."
                  )

my_assistant = client.beta.assistants.create(
    instructions=system_message,
    name="Isaac Assistant",
    tools=[{"type": "retrieval"}],#{"type": "code_interpreter"}],
    #model='gpt-3.5-turbo-1106',
    model="gpt-4-turbo-preview",
    # model=fine_tuned_model_id,
    file_ids=files_to_retrieve
)

# show_json(assistant)

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

prompt_answers_2 = {}

prompt_list = describe_prompt_list + topic_prompt_list + factual_prompt_list

prompt_list = prompt_list
i = 1
for prompt in prompt_list:
    print("i: ",str(i)," of ",str(len(prompt_list)))
    i += 1

    thread = client.beta.threads.create()
    # show_json(thread)

    print(prompt)

    prompt_message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
        file_ids=files_to_retrieve
    )

    chat_messages = []

    chat_messages.append({'role':'user','content':prompt})

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=my_assistant.id
    )

    run = wait_on_run(run, thread)
    # show_json(run)

    if run.status == 'completed':
      response_messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
      # show_json(messages)

      m = []
      for y in response_messages:
          m.append(y)
      m.reverse()
      answer = ""
      for x in m:
          # print(x)
          print('IsaacBot: ' if x.assistant_id else 'user:     ', end='')
          print(x.content[0].text.value)
          print()
          if x.assistant_id:
              answer += ' '+x.content[0].text.value

      print('IsaacBotAnswer:', answer)

      chat_messages.append({'role':'IsaacBot:  ','content':answer})

      prompt_answers_2[prompt] = answer

      print()
      print()
    else:
      print(run.status)

In [ ]:
chat_messages[0]['content'].content[0].text.value

In [ ]:
for x in chat_messages:
  print(x.keys())
  print(x['role'])
  print('#'*60)
  print(x['content'])
  # print(x)
  print('#'*120)

## Save the conversation to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

chat_messages = []

chat_messages.append({"role":"system","content":system_message})

for k,v in prompt_answers.items():
    chat_messages.append({"role":"user","content":k})
    chat_messages.append({"role":"assistant","content":v})

json_string = json.dumps(chat_messages)

conversation_name = "april-5-62-autobio-prompt-answers-gpt4-RAG-journal"

print("Conversation name:",conversation_name)

conversation_name_check = input("Confirm conversation name (Y) or provide a name for the conversation: ")
# Validate the conversation name

if not conversation_name_check.lower() == "y":
    conversation_name = conversation_name_check

while " " in conversation_name:
  print("Invalid conversation name. Please enter a name without spaces.")
  conversation_name = input("Provide a name for the conversation: ")

import datetime
eastern_now_str = datetime.datetime.utcnow().astimezone(datetime.timezone(datetime.timedelta(hours=-4))).strftime("%Y_%m_%d_%H:%M:%S")
conversation_date = eastern_now_str

# Create the file name.
file_name = f"{conversation_name}_{conversation_date}"

# Save the JSON string to a file in Google Drive.
with open(f"/content/drive/MyDrive/IsaacBotLogs/{file_name}.json", "w") as f:
    f.write(json_string)

with open(f"/content/drive/MyDrive/IsaacBotLogs/{file_name}.txt", "w") as f:
    f.write(str(len(chat_messages))+ " messages\n\n")
    f.write(f"system:      {system_message}\n\n")
    for k,v in prompt_answers.items():
        f.write(f"user:      {k}\n")
        f.write(f"assistant: {v}\n\n")
    # f.write(str(len(chat_messages))+" messages\n\n")
    # for message in chat_messages:
    #     role = message["role"]
    #     content = message["content"]
    #     f.write(f"{role}: {content}\n")

In [ ]:
for k,v in prompt_answers.items():
    print(k)
    print(v)